In [1]:
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
from keras.models import model_from_json
import numpy as np
import os
import random
import tensorflow as tf
from PIL import Image  

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
im = Image.open('/content/drive/MyDrive/Alphaversion/house2.jpg')  
newsize = (400, 400) 
im = im.resize(newsize) 
im.save('/content/drive/MyDrive/Alphaversion/house.jpg')

In [3]:
# Get images
image = img_to_array(load_img('/content/drive/MyDrive/Alphaversion/woman.jpg'))
image = np.array(image, dtype=float)

In [7]:
X = rgb2lab(1.0/255*image)[:,:,0]
Y = rgb2lab(1.0/255*image)[:,:,1:]
Y /= 128
print(X.shape)
X = X.reshape(1, 400, 400, 1)
Y = Y.reshape(1, 400, 400, 2)
print(X.shape)

(400, 400)
(1, 400, 400, 1)


In [ ]:
# Building the neural network
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

In [ ]:
# Finish model
model.compile(optimizer='rmsprop',loss='mse')

In [ ]:
model.fit(x=X, 
    y=Y,
    batch_size=1,
    epochs=1000)

In [ ]:
print(model.evaluate(X, Y, batch_size=1))
output = model.predict(X)
output *= 128
# Output colorizations
cur = np.zeros((400, 400, 3))
cur[:,:,0] = X[0][:,:,0]
cur[:,:,1:] = output[0]
imsave("/content/drive/MyDrive/Alphaversion/img_result.png", lab2rgb(cur))
imsave("/content/drive/MyDrive/Alphaversion/img_gray_version.png", rgb2gray(lab2rgb(cur)))
print("Done")

In [ ]:
model_json = model.to_json()
with open("/content/drive/MyDrive/Alphaversion/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/drive/MyDrive/Alphaversion/model.h5")
print("Saved model to disk")

Saved model to disk


In [9]:
# load json and create model
json_file = open('/content/drive/MyDrive/Alphaversion/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/drive/MyDrive/Alphaversion/model.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10200529592437301568
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15703311680
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6840600161872044117
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]
